# Basketball analytics: distilling and summarizing Information

#### Due: May 11 at 10 pm

When analyzing data, setting a goal is often helpful. In this assignment, the focus is on understanding how NMF behaves, and further analyzing player data.

In all the problems below, take a step back and think about each procedure as a piece in a bigger puzzle of understanding the game of basketball and its players. This goal should guide any decisions we make, and insights we interpret.

## Preparing Data

In the previous notebook `07-Shooting-Pattern-Analysis`, we computed smoothed shot patterns for 362 players that played during 2016-17 regular season. Save the matrix `X` from Non-negative matrix factorization (NMF) section.

Please create this file from saving the appropriate variable into a picke file called `allpatterns2016-17.pkl`. After saving the file, you can load it via the following command:

In [ ]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import subprocess as sp
import pickle

import helper_basketball as h
import imp
imp.reload(h);

import seaborn as sns

In [ ]:
# created the pickle file in lecture 7
X = pickle.load(open('allpatterns2016-17.pkl', 'rb'))
X.shape

## Non-negative Matrix Factorization (NMF) notation

Non-negative matrix factorization was used on the smoothed shooting pattern data of around 360 players. The result was useful in
* Bases: Identifying modes of shooting style (number of modes was determined by `n_components` argument to `NMF` function)
* Coefficients: How each players shooting style could be expressed as a linear combination of these bases (matrix multiplication between the bases and coefficients achieve this)

Recall the following. Given some matrix $X$ is $p\times n$ matrix, NMF computes the following factorization:
$$ \min_{W,H} \| X - WH \|_F\\
\text{ subject to } W\geq 0,\ H\geq 0, $$
where $W$ is ${p\times r}$ matrix and $H$ is ${r\times n}$ matrix.


## Problem 1

__PSTAT 134 and 234__: Experiment with different number of `n_components` to change the number of bases vectors. Visualize the bases vectors.

What value of $r$ seem to be too small? (`r` is too small to represent diversity of shooting modes) What value of $r$ seem to be too large? (`r` is too large and some bases seem to be duplicated). Note that, if a basis were a perfect duplicate of another (they will not be, but could be similar), you would use one basis instead of two.

In [ ]:
import sklearn.decomposition as skld
from ipywidgets import interact, FloatSlider, Dropdown, Button

comp_dd = dict(zip(range(1,81),range(1,81)))
default_component = 10

component_menu = Dropdown(options = comp_dd, value = default_component)

fetch_button = Button(description='Get Data!', icon='check')
display(component_menu, fetch_button)

xedges = (np.linspace(start=-25, stop=25, num=151, dtype=np.float)) * 12
yedges = (np.linspace(start= -4, stop=31, num=106, dtype=np.float)) * 12
## Non-negative Matrix Factorization
def non_negative_marix_decomp(n_components,train_data):
    import sklearn.decomposition as skld
    model = skld.NMF(n_components=n_components, init='nndsvda', max_iter=500, random_state=0)
    W = model.fit_transform(train_data)
    H = model.components_
    nmf = (W,H)
    return(nmf)

def get_data(change):
    print('Number of bases (r):', component_menu.value)
    
    W, H = non_negative_marix_decomp(n_components = component_menu.value, train_data = X)

    p_w, r = W.shape
    r = int(round(r/2)) if ((int(r) % 2) == 0) else int(r/2) + 1
    
    fig, ax = plt.subplots(r, 2, figsize=(20,int(8*r)))
    print
    helper = ax.flatten() if ((component_menu.value % 2) == 0) else ax.flatten()[:-1]
    
    for i, axi in enumerate(helper):
        h.plot_shotchart(W[:,i], xedges, yedges, ax=axi)
        axi.set_title('NMF component ' + str(i))
    

fetch_button.on_click(get_data)

4 might be too little

## Problem 2

__PSTAT 134 and 234__: In the previous question, NMF gave us a set of bases to describe each player. So, the comparison is through a standard set of shooting styles. We may also approach the comparison more directly.

* In this problem, we compare of players' shooting styles to each other directly. What we are interested in is pairwise correlation between shooting patterns. Let $X_i$ represent the column in the smoothed shooting pattern for player $i$. Then, we want to compute   
    $$ R = [\text{Cor} (X_i, X_j)]_{i,j} $$ for all player combinations $i,j\in\{1,2,\dots,362\}$. What is the correct orientation of matrix $X$? What should be the dimension of matrix $R$?   
    _Note: if your command is not running properly, you may be running into the issue of using too much memory, and your notebook session is rebooted by the server as a result._
    
* Visualize matrix $R$ with [seaborn.heatmap](https://seaborn.pydata.org/generated/seaborn.heatmap.html) function.

* Identify 2 pairs of players with highest similarities (positive correlation) and 2 pairs with lowest similarity (negative correlation). Plot their shooting pattern. What do you observe?

keep X as is  
362x362

In [ ]:
R = np.corrcoef(X, rowvar=0)
R.shape

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(15, 15)
#sns.set()
sns.heatmap(R, cmap="Blues_r")

In [ ]:
flat = R.flatten()
flat.sort()
flat = np.delete(flat, np.array(range(len(flat)-362,len(flat))))
print("minimum correlation value: ", flat[0])
print("maximum correlation value: ", flat[-1])

In [ ]:
np.where(R == flat[-1])

In [ ]:
np.where(R == flat[0])

In [ ]:
allshots = pickle.load(open('allshots2016-17.pkl', 'rb'))
player_ID = allshots.PlayerID.unique()

## get all players
params = {'LeagueID':'00', 'Season': '2016-17', 'IsOnlyCurrentSeason': '0'}
players = h.get_nba_data('commonallplayers', params)

In [ ]:
print("IDs of players with highest correlation: ",player_ID[283],',',player_ID[235]) 
print("IDs of players with lowest correlation: ",player_ID[23],',',player_ID[251]) 

In [ ]:
print("Players with highest correlation:",players.query('PERSON_ID == 2216')['DISPLAY_FIRST_LAST'].item(),',',
      players.query('PERSON_ID == 1626257')['DISPLAY_FIRST_LAST'].item()) 
print("Players with lowest correlation:",players.query('PERSON_ID == 203115')['DISPLAY_FIRST_LAST'].item(),',',
      players.query('PERSON_ID == 203488')['DISPLAY_FIRST_LAST'].item()) 

In [ ]:
# Plot the players with the highest correlation which means that they have the most similar shooting patterns
fig, ax = plt.subplots(1,2, figsize=(20,60))

h.plot_shotchart(X[:,283], xedges, yedges, ax=ax[0]) 
h.plot_shotchart(X[:,235], xedges, yedges, ax=ax[1])
ax[0].set_title('Zach Randolph Shooting Pattern')
ax[1].set_title('Salah Mejri Shooting Pattern')

In [ ]:
# Plot the players with the lowest correlation which means that they have the least similar shooting patterns
fig, ax = plt.subplots(1,2, figsize=(20,60))

h.plot_shotchart(X[:,23], xedges, yedges, ax=ax[0]) 
h.plot_shotchart(X[:,251], xedges, yedges, ax=ax[1])
ax[0].set_title('Will Barton Shooting Pattern')
ax[1].set_title('Mike Muscala Shooting Pattern')

## Problem 3

__PSTAT 134 and 234__: How would you use the coefficients matrix $H$ from NMF  or the correlation matrix $R$ (computed above) to differentiate between types of players? Consider what the coefficients represent, and how you can use them to discriminate player types.

Give your thought process, reasoning for your chosen method, and the results. Do they look reasonable? Do you expect any of the comparison to be similar to any of the [figures here](https://fastbreakdata.com/classifying-the-modern-nba-player-with-machine-learning-539da03bb824)? Why, or why not? Can you verify your intuition?

## Problem 4

__PSTAT 134 and 234__: Suppose you are in charge of a basketball team. How would you use this information? How would you use what you have learned from analyzing the data, and what other questions would you like to answer with further analysis.